<a href="https://colab.research.google.com/github/aankit/nycdoe_space_analysis/blob/master/DOE_Space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Understanding the Physical Space of the Largest School District in the United States

New York City has the largest school district in the country based on the number of students it serves. The COVID-19 pandemic has asked us all to be more cognizant of our physical space whether it be staying 6 feet apart in public or staying sane at home, especially if you live in a tiny NYC apartment.

###How much physical space does each student have in NYC schools? How does the space per student differ between schools and is there a potential disparate impact? 

Feel free to reach out to me at hi@aankit.com if you have questions or thoughts.

###Question 1: Does publicly available data about the physical space of schools exist?

###Answer: Yes! Below I point out and load up some data sources for my analysis.

First I'm going to install and load some helpful tools :)

In [ ]:
!pip install geopandas

In [ ]:
from __future__ import print_function
import requests
import pandas as pd
import io
import geopandas as gpd

DOE Building Space Usage is available on the NYC Open Data Portal [here](https://data.cityofnewyork.us/Education/DOE-Building-Space-Usage/wavz-fkw8). I'm using the CSV because the [JSON API endpoint](https://data.cityofnewyork.us/resource/wavz-fkw8.json) only yields 1000 rows at a time I don't feel like writing a loop to page through.

In [ ]:
response = requests.get("https://data.cityofnewyork.us/api/views/wavz-fkw8/rows.csv?accessType=DOWNLOAD")
school_space = pd.read_csv(io.StringIO(response.text))

Attendance history will give us a somewhat accurate picture of how many students are in each building.

In [ ]:
response = requests.get("https://data.cityofnewyork.us/api/views/vww9-qguh/rows.csv?accessType=DOWNLOAD")
attendance_history = pd.read_csv(io.StringIO(response.text))

We will need more information schools - like what grades they serve and where they are located for a proper analysis. 

DOE School Information is available via a file called "LCGMS" [here](https://data.cityofnewyork.us/Education/LCGMS-DOE-School-Information-Report/3bkj-34v2). I'm using the [CSV with additional geocoded fields](https://data.cityofnewyork.us/api/views/3bkj-34v2/files/56813139-9b9d-44fb-b81d-068553b7a9b7?download=true&filename=LCGMS_SchoolData(additional%20geocoded%20fields%20added).csv)

In [ ]:
response = requests.get("https://data.cityofnewyork.us/api/views/3bkj-34v2/files/56813139-9b9d-44fb-b81d-068553b7a9b7?download=true&filename=LCGMS_SchoolData(additional geocoded fields added).csv")
lcgms = pd.read_csv(io.StringIO(response.text))

And finally we will want to look at all of this on a map, so let's load a geospatial file of school zones from [here](https://data.cityofnewyork.us/Education/2019-2020-School-Zones-Elementary-/kuk3-ypca). I'm using the shapefile.

This is only relevant for elementary schools (and maybe some middle schools).

In [ ]:
response = requests.get("https://data.cityofnewyork.us/api/geospatial/kuk3-ypca?method=export&format=GeoJSON")
school_zones = pd.DataFrame(response.json())

###Question 2: What areas of a school are instructional areas?

The data dictionary provided by the School Construction Authority on the  NYC Open Data Portal is helpful. It tells us that the DOE Building Space data we pulled in and named `school_space` has a `Room Function` column, or field. Let's get a count of the values in the `Room Function` field.

In [ ]:
school_space["Room Function"].value_counts()

REGULAR CLASSROOM                70581
STORAGE ROOM                     46699
OTHER OFFICE                     36936
GENERAL BUILDING SUPPORT         31838
REGULAR CLASSROOM - MS GRADES    19807
                                 ...  
ELEVENTH GRADE                      94
GYM/AUD/CAFETERIA                   89
DRAFTING ROOM                       73
TWELFTH GRADE                       51
NEST NINTH-TWELFTH GRADE            13
Name: Room Function, Length: 109, dtype: int64

Based on this quick overview, if `Room Function` contains the word "classroom" or "grade" it can probably be classified as an instructional space.

In [ ]:
instructional_school_space = school_space[school_space["Room Function"].str.contains("CLASSROOM|GRADE", na=False)]
instructional_school_space["Room Function"].value_counts()

REGULAR CLASSROOM                   70581
REGULAR CLASSROOM - MS GRADES       19807
D75 SPED CLASSROOM                  15482
NON-D75 SPED CLASSROOM              15481
FIRST GRADE                         13521
SECOND GRADE                        12696
THIRD GRADE                         12193
FOURTH GRADE                        11622
FIFTH GRADE                         11369
ICT - ELEMENTARY SCHOOL GRADES      10215
REGULAR CLASSROOM - HS GRADES        3813
MULTI-PURPOSE CLASSROOM              2544
SCIENCE CLASSROOM FOR PS             2028
SIXTH GRADE                          1644
ICT - MIDDLE SCHOOL GRADES           1558
SEVENTH GRADE                        1526
EIGHTH GRADE                         1482
MULTI-PURPOSE NON CLASSROOM          1177
NEST FIRST-THIRD GRADE                452
ICT - HIGH SCHOOL GRADES              424
NEST SIXTH-EIGHTH GRADE               407
NEST FOURTH-FIFTH GRADE               248
NINTH GRADE                           236
HORIZON SECOND-TWELFTH GRADE      

Let's take a look at the square footage of this instructional space. This view is aggregating across schools. The next step is to drill down to the school level before we link schools to the communities they serve.

In [ ]:
instructional_school_space.groupby("Room Function").sum()

,Length,Width,Area
Room Function,,,
D75 SPED CLASSROOM,372679.0,345510.0,8302789.0
EIGHTH GRADE,37973.0,40526.0,1031246.0
ELEVENTH GRADE,2529.0,2424.0,64696.0
FIFTH GRADE,281419.0,300719.0,7362002.0
FIRST GRADE,339764.0,352957.0,8773473.0
FOURTH GRADE,287273.0,306075.0,7479172.0
HORIZON KINDERGARTEN-FIRST GRADE,2509.0,2243.0,58305.0
HORIZON SECOND-TWELFTH GRADE,5293.0,5087.0,127132.0
ICT - ELEMENTARY SCHOOL GRADES,255937.0,269540.0,6657939.0


###Question 3: Can we determine each school's square footage per person?

The `Contributing 20+ Total Days` field in `attendance_history` tells us how many students spend 20 or more days at the each school. This slightly inflates attendance numbers, but also captures the fact that NYC schools deal with fluctuating attendance due to students joining and leaving.

We can focus on the most recent year's attendance, but there are numbers going back to 2013-14 allowing for historical analysis. Finally the data also provides attendance by grade and demographic category, but again for simplicity we can look at the aggregate of all students for the school.

In [150]:
attendance = attendance_history[(attendance_history["Year"].eq("2018-19")) & (attendance_history["Grade"].eq("All Grades")) & (attendance_history["Demographic Category"].eq("All Students"))]
attendance["DBN"].head(15)

06M366    1
26Q205    1
23K284    1
30Q085    1
18K235    1
         ..
08X559    1
17K531    1
26Q133    1
09X231    1
08X376    1
Name: DBN, Length: 1526, dtype: int64

Let's take some columns from `attendance_history` and combine it with the `instructional_school_space` and do a calculation of square foot per student.

In [151]:
instructional_space_by_school = instructional_school_space.groupby("Org Code").sum()
iss_lcgms = pd.merge(instructional_space_by_school, lcgms, left_on="Org Code", right_on="Location Code")
all_data = pd.merge(iss_lcgms, attendance, left_on="ATS System Code", right_on="DBN")
all_data.head()


,Length,Width,Area,ATS System Code,Location Code,Location Name,BEDS Number,Managed By Name,Location Type Description,Location Category Description,Grades,Grades Final,Open Date,Status Description,Building Code,Primary Address,City,State Code,Zip,Borough Block Lot,Census Tract,Community District,Council District,NTA,NTA_Name,Principal Name,Principal Title,Principal Phone Number,Fax Number,Geographical District Code,Administrative District Code,Administrative District Location Code,Administrative District Name,Superintendent,Superintendent Location Code,Community School Sup Name,BFSC Location Code,BFSC Director Name,BFSC Director Title,BFSC Director Phone,HighSchool Network Location Code,HighSchool Network Name,HighSchool Network Superintendent,Borough,BIN,Latitude,Longitude,DBN,School Name,Grade,Year,Demographic Category,Demographic Variable,# Total Days,# Days Absent,# Days Present,% Attendance,# Contributing 20+ Total Days,# Chronically Absent,% Chronically Absent
0,5203.0,4881.0,127110.0,15K001,K001,P.S. 001 The Bergen,3.320000e+11,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",Jul 1 1965,Open,K001,309 47 STREET,BROOKLYN,NY,11220,3007550022,8000.0,307,38,BK32,Sunset Park West,Arlene Ramos,PRINCIPAL,718-567-7661,718-567-9771,15,15,K815,COMMUNITY SCHOOL DISTRICT 15,"SKOP, ANITA",K815,"SKOP, ANITA",KFSN,"FITZGERALD, BERNADETTE",FSC Executive Director,718-935-3954,NaN,NaN,NaN,BROOKLYN,,40.649042,-74.012289,15K001,P.S. 001 The Bergen,All Grades,2018-19,All Students,All Students,212999,11836,201163,94.4,1242,215,17.3
1,3850.0,2885.0,87958.0,17K002,K002,Parkside Preparatory Academy,3.320000e+11,DOE,General Academic,Junior High-Intermediate-Middle,"06,07,08,SE","06,07,08",Jul 1 1994,Open,K002,655 PARKSIDE AVENUE,BROOKLYN,NY,11226,3050500051,80200.0,309,40,BK60,Prospect Lefferts Gardens-Wingate,ADRIENNE SPENCER,PRINCIPAL,718-462-6992,718-284-7717,17,17,K817,COMMUNITY SCHOOL DISTRICT 17,"ELLIS, CLARENCE",K817,"ELLIS, CLARENCE",KFSS,Mauriciere de Govia,FSC Executive Director,718-240-3677,NaN,NaN,NaN,BROOKLYN,,40.656183,-73.951583,17K002,Parkside Preparatory Academy,All Grades,2018-19,All Students,All Students,84480,5067,79413,94.0,495,97,19.6
2,2069.0,3007.0,65252.0,13K003,K003,P.S. 003 The Bedford Village,3.310000e+11,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",Jul 1 1950,Open,K003,50 JEFFERSON AVENUE,BROOKLYN,NY,11216,3019980001,22700.0,303,36,BK69,Clinton Hill,KRISTINA BEECHER,PRINCIPAL,718-622-2960,718-623-3193,13,13,K813,COMMUNITY SCHOOL DISTRICT 13,"FREEMAN, BARBARA",K813,"FREEMAN, BARBARA",KFSN,"FITZGERALD, BERNADETTE",FSC Executive Director,718-935-3954,NaN,NaN,NaN,BROOKLYN,,40.682663,-73.955338,13K003,P.S. 003 The Bedford Village,All Grades,2018-19,All Students,All Students,64576,6887,57689,89.3,392,169,43.1
3,6980.0,7198.0,166372.0,75K004,K004,P.S. K004,3.080000e+11,DOE,Special Education,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05,06,07,SE",Jul 1 1999,Open,K843,530 STANLEY AVENUE,BROOKLYN,NY,11207,3043690001,110400.0,305,42,BK82,East New York,DEBORAH EVANS,PRINCIPAL,718-498-6680,718-927-3554,19,75,DA75,CITYWIDE SPECIAL EDUCATION,"LOUISSAINT, KETLER",DA75,"LOUISSAINT, KETLER",D075,Tillman Roberto,Cluster Leader,212-802-1610,NaN,NaN,NaN,BROOKLYN,,40.657212,-73.888852,75K004,P.S. K004,All Grades,2018-19,All Students,All Students,58612,7686,50926,86.9,350,185,52.9
4,2171.0,2871.0,66941.0,16K005,K005,P.S. 005 Dr. Ronald Mcnair,3.320000e+11,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",Jul 1 1966,Open,K005,820 HANCOCK STREET,BROOKLYN,NY,11233,3014900001,37700.0,303,41,BK35,Stuyvesant Heights,LENA GATES,PRINCIPAL,718-218-2444,718-218-2445,16,16,K816,COMMUNITY SCHOOL DISTRICT 16,"AMON, RAHESHA",K816,"AMON, RAHESHA",KFSN,"FITZGERALD, BERNADETTE",FSC Executive Director,718-935-3954,NaN,NaN,NaN,BROOKLYN,,40.685695,-73.922463,16K005,P.S. 005 Dr. Ronald McNair,All Grades,2018-19,All Students,All Students,39107,5288,3381

This is a lot of really good data in one place!

Stay focused :) Let's calculate a column, Sq Ft. Per Student, for each school using this formula: `Area / (# Contributing 20+ Total Days * % Attendance)`

In [189]:
#num_contributing_20plus_total_days
all_data.columns = all_data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%', 'perc').str.replace('#', 'num').str.replace('+', 'plus')
all_data["sq_ft_per_student"] = all_data.area / (all_data.num_contributing_20plus_total_days.astype(int) * (all_data.perc_attendance.astype(float)/100))
all_data[["dbn", "sq_ft_per_student"]].sort_values("sq_ft_per_student").head(25)

,dbn,sq_ft_per_student
1462,11X481,19.792045
1504,08X583,27.682813
455,22K889,30.367123
1040,26Q376,31.186364
607,02M297,32.050020
1041,27Q377,32.638033
1038,30Q361,32.843453
1037,29Q360,33.195165
1144,31R068,34.899139
310,19K422,44.836453


In [191]:
all_data.to_excel("space_analysis.xlsx")
from google.colab import files
files.download('space_analysis.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Question 4: Can we map each school's square footage per person to get an idea of the impact on different communities? 